In [ ]:
!python -m pip install mysql-connector-python

In [1]:
import pandas as pd
import pickle
import re
import mysql.connector
import annotation_function
import connection_database

## Get emails of interest

In [2]:
with open('list_ids_for_annotation.pkl', 'rb') as f:
    ids_list = pickle.load(f)

In [3]:
with open('dutch_cleaned_emails_id_body_org.pkl', 'rb') as f:
    all_emails = pickle.load(f)

In [4]:
# Keep only emails that will be used for annotation
mask = all_emails['id'].isin(ids_list)
emails = all_emails.loc[mask]

## Get action from customer support

In [9]:
mydb = connection_database.connect_db()
cursor = mydb.cursor()

command = """SELECT emails.id, tickets.ticket_status_id , tickets.action_id  FROM emails 
JOIN tasks ON emails.task_id=tasks.id 
JOIN tickets ON emails.ticket_id=tickets.id
WHERE organization_id IN 
(SELECT id FROM organizations WHERE default_pgrefid IS NULL)
AND emails.from NOT LIKE '%@closure-services.nl%'
AND emails.is_auto_receival=0
AND emails.subject NOT LIKE "%ntvangst bevestiging%" 
AND emails.created_at > 20181231
AND tickets.is_automated = 0 
AND tickets.ticket_status_id != 6"""

cursor.execute(command)
table_rows = cursor.fetchall()
actions = pd.DataFrame(table_rows, columns=cursor.column_names)

In [10]:
emails = emails.merge(actions, on="id")

## Get label

In [13]:
emails["label"] = emails.apply(lambda x: annotation_function.get_label(x["ticket_status_id"], x["action_id"]), axis = 1)

## Modify label for charities and journals

In [14]:
command = """SELECT id, organization_sector_id FROM organizations"""

cursor.execute(command)
table_rows = cursor.fetchall()
org_sector = pd.DataFrame(table_rows, columns=cursor.column_names)

In [15]:
# Get IDs of charities and journals
org_id_char_jour = org_sector[(org_sector.organization_sector_id == 3) | (org_sector.organization_sector_id == 7)].id.tolist()

In [17]:
emails["label_modified"] = emails.apply(lambda x: annotation_function.fix_label_charity(x["organization_id"], x["label"], x["only_body"], org_id_char_jour), axis = 1)

## Get automatic emails

In [20]:
emails["is_auto"] = emails["only_body"].apply(annotation_function.find_auto_emails)
auto_emails = emails[emails.is_auto == 1]
auto_id = auto_emails.id.tolist()

In [26]:
# Save emails that are automatically sent (label 3)
with open('list_ids_label3.pkl', 'wb') as f:
    pickle.dump(auto_id, f)

## Save customer support labels (without automatic emails)

In [26]:
# Remove automatic emails
emails = emails[emails.is_auto == 0]

In [28]:
with open('label_CS_actions.pkl', 'wb') as f:
    pickle.dump(emails[["id", "label_modified"]], f)